<a href="https://colab.research.google.com/github/MatveyOreshkinn/Minor/blob/main/%D0%9B%D0%A01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:

from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import pandas as pd
import numpy as np
import re

from google.colab import drive
import warnings


warnings.filterwarnings("ignore")
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
log_pattern = r'(\S+) - - \[(.*?)\] "(\S+) (\S+) (\S+\s\S+)" (\d+) (\d+) (\S+)'
# Создаем пустой список для хранения распарсенных данных
parsed_data = []

# Открываем файл и читаем построчно
with open('/content/drive/MyDrive/access.log', 'r') as file: # Убедитесь, что указали правильный путь к файлу
    for line in file:
        # Ищем совпадения с нашим шаблоном в каждой строке
        match = re.match(log_pattern, line)
        if match:
            # Если строка соответствует шаблону, извлекаем группы
            parsed_data.append(match.groups())

# Определяем названия колонок для будущего DataFrame
column_names = ['ip', 'timestamp', 'http_method', 'endpoint', 'http_version', 'status_code', 'response_size', 'session_id']

# Создаем DataFrame
df = pd.DataFrame(parsed_data, columns=column_names)
df.head()

,ip,timestamp,http_method,endpoint,http_version,status_code,response_size,session_id
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,HTTP 1.1,200,4407,ID4887
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,HTTP 1.1,200,4025,ID4887
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,HTTP 1.1,200,3482,ID4887
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,HTTP 1.1,200,4904,ID4887
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,HTTP 1.1,200,2104,ID4887


In [5]:
df.describe()

,ip,timestamp,http_method,endpoint,http_version,status_code,response_size,session_id
count,239202,239202,239202,239202,239202,239202,239202,239202
unique,50850,239202,2,102,1,1,3000,50850
top,143.167.87.213,31/Jan/2015:23:59:57 +03:00,POST,/catalog.phtml,HTTP 1.1,200,4026,ID22347
freq,39,1,219864,110410,239202,239202,112,39


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239202 entries, 0 to 239201
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ip             239202 non-null  object
 1   timestamp      239202 non-null  object
 2   http_method    239202 non-null  object
 3   endpoint       239202 non-null  object
 4   http_version   239202 non-null  object
 5   status_code    239202 non-null  object
 6   response_size  239202 non-null  object
 7   session_id     239202 non-null  object
dtypes: object(8)
memory usage: 14.6+ MB


In [8]:
df['http_version'].value_counts()  # все версия протокола одинаковые

,count
http_version,
HTTP 1.1,239202


In [9]:
df['status_code'].value_counts()  # все ответы серверы были успешными

,count
status_code,
200,239202


In [14]:
df.drop(['http_version', 'status_code'], inplace=True, axis=1)

df.head()

,ip,timestamp,http_method,endpoint,response_size,session_id
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,4407,ID4887
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,4025,ID4887
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,3482,ID4887
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,4904,ID4887
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,2104,ID4887


In [11]:
df['http_method'].value_counts()

,count
http_method,
POST,219864
GET,19338


In [16]:
df['session_id'].value_counts()

,count
session_id,
ID22347,39
ID52666,35
ID21489,33
ID38085,33
ID13435,33
...,...
ID55685,1
ID55686,1
ID4831,1


# Кластеризация


In [19]:
# кодирование ip
df[['octet1', 'octet2', 'octet3', 'octet4']] = df['ip'].str.split('.', expand=True)

df.head()

,ip,timestamp,http_method,endpoint,response_size,session_id,octet1,octet2,octet3,octet4
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,4407,ID4887,179,203,10,126
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,4025,ID4887,179,203,10,126
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,3482,ID4887,179,203,10,126
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,4904,ID4887,179,203,10,126
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,2104,ID4887,179,203,10,126


In [29]:
# кодирование методов

encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[['http_method']])

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['http_method']))
df = pd.concat([df, encoded_df], axis=1)

df.head()

,ip,timestamp,http_method,endpoint,response_size,session_id,octet1,octet2,octet3,octet4,http_method_GET,http_method_POST
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,4407,ID4887,179,203,10,126,0.0,1.0
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,4025,ID4887,179,203,10,126,0.0,1.0
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,3482,ID4887,179,203,10,126,0.0,1.0
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,4904,ID4887,179,203,10,126,0.0,1.0
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,2104,ID4887,179,203,10,126,0.0,1.0


In [31]:
# Кодируем session_id
frequency_map = df['session_id'].value_counts().to_dict()
df['session_freq'] = df['session_id'].map(frequency_map)

df.head()

,ip,timestamp,http_method,endpoint,response_size,session_id,octet1,octet2,octet3,octet4,http_method_GET,http_method_POST,session_freq
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,4407,ID4887,179,203,10,126,0.0,1.0,9
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,4025,ID4887,179,203,10,126,0.0,1.0,9
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,3482,ID4887,179,203,10,126,0.0,1.0,9
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,4904,ID4887,179,203,10,126,0.0,1.0,9
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,2104,ID4887,179,203,10,126,0.0,1.0,9


In [32]:
# Кодируем endpoint
frequency_map = df['endpoint'].value_counts().to_dict()
df['endpoint_freq'] = df['endpoint'].map(frequency_map)

df.head()

,ip,timestamp,http_method,endpoint,response_size,session_id,octet1,octet2,octet3,octet4,http_method_GET,http_method_POST,session_freq,endpoint_freq
0,179.203.10.126,1/Jan/2015:00:00:02 +03:00,POST,/catalog.phtml,4407,ID4887,179,203,10,126,0.0,1.0,9,110410
1,179.203.10.126,1/Jan/2015:00:00:13 +03:00,POST,/search.phtml,4025,ID4887,179,203,10,126,0.0,1.0,9,109454
2,179.203.10.126,1/Jan/2015:00:00:15 +03:00,POST,/search.phtml,3482,ID4887,179,203,10,126,0.0,1.0,9,109454
3,179.203.10.126,1/Jan/2015:00:00:24 +03:00,POST,/catalog.phtml,4904,ID4887,179,203,10,126,0.0,1.0,9,110410
4,179.203.10.126,1/Jan/2015:00:00:30 +03:00,POST,/search.phtml,2104,ID4887,179,203,10,126,0.0,1.0,9,109454


In [40]:
# кодируем timestamp

df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z')


df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second
df['day'] = df['timestamp'].dt.day
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year


df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)


df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)


df['second_sin'] = np.sin(2 * np.pi * df['second'] / 60)
df['second_cos'] = np.cos(2 * np.pi * df['second'] / 60)


days_in_month = df['timestamp'].dt.days_in_month
df['day_sin'] = np.sin(2 * np.pi * df['day'] / days_in_month)
df['day_cos'] = np.cos(2 * np.pi * df['day'] / days_in_month)


df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)


df = df.drop(['hour', 'minute', 'second', 'day', 'month'], axis=1)
df.head()


,ip,timestamp,http_method,endpoint,response_size,session_id,octet1,octet2,octet3,octet4,...,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos,day_sin,day_cos,month_sin,month_cos
0,179.203.10.126,2015-01-01 00:00:02+03:00,POST,/catalog.phtml,4407,ID4887,179,203,10,126,...,0.0,1.0,0.0,1.0,2.079117e-01,9.781476e-01,0.201299,0.97953,0.5,0.866025
1,179.203.10.126,2015-01-01 00:00:13+03:00,POST,/search.phtml,4025,ID4887,179,203,10,126,...,0.0,1.0,0.0,1.0,9.781476e-01,2.079117e-01,0.201299,0.97953,0.5,0.866025
2,179.203.10.126,2015-01-01 00:00:15+03:00,POST,/search.phtml,3482,ID4887,179,203,10,126,...,0.0,1.0,0.0,1.0,1.000000e+00,2.832769e-16,0.201299,0.97953,0.5,0.866025
3,179.203.10.126,2015-01-01 00:00:24+03:00,POST,/catalog.phtml,4904,ID4887,179,203,10,126,...,0.0,1.0,0.0,1.0,5.877853e-01,-8.090170e-01,0.201299,0.97953,0.5,0.866025
4,179.203.10.126,2015-01-01 00:00:30+03:00,POST,/search.phtml,2104,ID4887,179,203,10,126,...,0.0,1.0,0.0,1.0,5.665539e-16,-1.000000e+00,0.201299,0.97953,0.5,0.866025
